## Импорт библиотек

In [39]:
import pandas as pd
import os
import warnings
import numpy as np
import re
import json

from functools import reduce
from datetime import datetime
from datetime import date

warnings.filterwarnings("ignore")

## Определение функций

## Установка параметров

In [40]:
marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
                     'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']
data_columns = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 
                'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                'productionDate', 'sell_id', 'vehicleTransmission', 'vendor', 
                'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']
outer_datasets = ['all_auto_ru_09_09_2020.csv',
                     'all_car_27_08_2021_data_12_brands.csv',
                     'data1.csv',
                     'parsing_auto_ru.csv',
                     'train_df_full_part1.csv']

### Сокращение (перезапись только брендов из test)

In [41]:
# for df_name,pars_date in  zip(outer_datasets,parsing_date):
#     try:
#         df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
#     except Exception:
#         df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
#     df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
#     file_name = 'auto_ru_' + pars_date + '.csv'
#     print(f'./Project_7_data/{file_name}')
#     df.to_csv(f'./Project_7_data/{file_name}',index=False)

### Определение даты каждого датсета/ курса на эту дату

In [42]:
parsing_date = ['2020_09_09','2021_08_29','2020_10_19','2021_08_13','2021_10_01']
currency_rate = [1,1,1,1,1]
basket_currency_rate_ = [1,1,1,1,1]

In [43]:
datasets_to_train = []
for df_name,pars_date in  zip(outer_datasets,parsing_date):
    file_name = 'auto_ru_' + pars_date + '.csv'
    datasets_to_train.append(file_name)

In [44]:
datasets_to_train

['auto_ru_2020_09_09.csv',
 'auto_ru_2021_08_29.csv',
 'auto_ru_2020_10_19.csv',
 'auto_ru_2021_08_13.csv',
 'auto_ru_2021_10_01.csv']

### Корреспонденция колонок

### унификация с test  - auto_ru_2020_09_09.csv  с учетом data_columns

In [45]:
test = pd.read_csv('./Project_7_data/test.csv',low_memory=False)

In [46]:
df= pd.read_csv('./Project_7_data/' + datasets_to_train[0],low_memory=False)

In [47]:
test.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

In [48]:
df.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'],
      dtype='object')

In [49]:
test_0909_uni = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 
                'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                'productionDate', 'vehicleTransmission', 'vendor', 
                'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']

## ============================================

#### bodyType

In [50]:
%%time

def bodyType_unification(x):
    res=[]
    x = x.lower() if type(x) == str else x  # <================
    try:
        for body_type in test.bodyType.unique():
            if body_type in x:
                res.append(body_type)
    except Exception: return x
    return max(res)

df.dropna(subset=['bodyType'],inplace=True)
lambda x: x.lower() if x else x
df.bodyType = df.bodyType.apply(bodyType_unification)

Wall time: 1min 33s


In [51]:
set(df.bodyType.unique()) == set(test.bodyType.unique())

True

#### brand

In [52]:
set(df.brand.unique()) == set(test.brand.unique())

True

#### color 

In [53]:
color_codes = {
    '040001': 'чёрный',
    'FAFBFB': 'белый', 
    '0000CC': 'синий', 
    '200204': 'коричневый', 
    'EE1D19': 'красный', 
    'CACECB': 'серый',
    'C49648': 'бежевый', 
    '97948F': 'серебристый', 
    'FFD600': 'жёлтый', 
    'FF8649': 'оранжевый', 
    '22A0F8': 'голубой',
    'FFC0CB': 'розовый', 
    'DEA522': 'золотистый', 
    '007F00': 'зелёный', 
    '660099': 'пурпурный',
    '4A2197': 'фиолетовый'}
df.color = df.color.map(color_codes)

In [54]:
set(test.color.unique()) == set(df.color.unique())

True

#### engineDisplacement

In [55]:
test.engineDisplacement = test.engineDisplacement.apply(
    lambda x: float(x[:-4]) if x[:-4] else np.NaN)   

In [56]:
def engineDisplacement_unification(x):
    x = float(re.sub("[^\d.]", r'', x)) if re.sub("[^\d.]", r'', x) else 0
    if x >= 7: x = 0
#     if 10 <= x < 100: return x/10
#     if 100 <= x: return x/100
    return x

df.engineDisplacement = df.engineDisplacement.apply(engineDisplacement_unification)

#### enginePower

In [57]:
test.enginePower = test.enginePower.apply(lambda x: int(x[:-4]))

In [58]:
df.enginePower = df.enginePower.astype(int)

#### equipment_dict

In [59]:
test.equipment_dict = test.equipment_dict.apply(lambda x: list(json.loads(x).keys()) if x==x else [])

In [60]:
def train_options(x):
    point = "'available_options': "
    start = x.find(point)+len(point)+2
    finish = x.find("]",start) - 1
    return x[start:finish].split("', '")

df.Комплектация = df.Комплектация.apply(train_options)

#### fuelType

In [61]:
set(test.fuelType.unique()) == set(df.fuelType.unique())

True

#### mileage <===== ВСЕ ОК

#### modelDate

In [62]:
df.modelDate = df.modelDate.astype(int)

#### model_name

In [63]:
{1,2,3} - {1,2}

{3}

In [64]:
test_model = set(test.model_name.unique())

In [65]:
train_model = set(df.model.unique())

In [66]:
test_model - train_model

{'300ZX',
 'ARTEON',
 'ATLAS',
 'COROLLA_II',
 'GOLF_R32',
 'IST',
 'LANCER_RALLIART',
 'LM',
 'OTTI',
 'POLO_GTI',
 'PORTE',
 'PRESEA',
 'RSQ3',
 'SPARKY',
 'TORNEO',
 'WILL_CYPHA',
 'X4_M',
 'Z',
 'ZEST'}

In [67]:
train_model - test_model

{'180SX',
 'BREVIS',
 'CELSIOR',
 'CLIPPER_RIO',
 'EDIX',
 'EQC',
 'KLUGER',
 'LANCER_CARGO',
 'LIFE',
 'M8',
 'MISTRAL',
 'MOCO',
 'N_ONE',
 'PAO',
 'PASSPORT',
 'PRIMASTAR',
 'Q3_SPORTBACK',
 'RC_F',
 'REGIUSACE',
 'ROOMY',
 'SLC_KLASSE_AMG',
 'SLK_KLASSE_AMG',
 'S_MX',
 'TACOMA',
 'TITAN',
 'TYPE_2',
 'TYPE_3',
 'W110',
 'W189'}

#### numberOfDoors

In [68]:
df.numberOfDoors = df.numberOfDoors.astype(int)

In [69]:
set(test.numberOfDoors.unique()) == set(df.numberOfDoors.unique())

True

#### productionDate

#### vehicleTransmission

In [70]:
transmission_dict = {
    'MECHANICAL':'механическая', 
    'AUTOMATIC':'автоматическая', 
    'ROBOT':'роботизированная', 
    'VARIATOR':'вариатор'}
df.vehicleTransmission = df.vehicleTransmission.map(transmission_dict)

In [71]:
set(test.vehicleTransmission.unique()) == set(df.vehicleTransmission.unique())

True

#### vendor 

In [76]:
vendor_dict = {
    'AUDI':'EUROPEAN',
    'BMW':'EUROPEAN',
    'HONDA':'JAPANESE',
    'INFINITI':'JAPANESE',
    'LEXUS':'JAPANESE',
    'MERCEDES':'EUROPEAN',
    'MITSUBISHI':'JAPANESE',
    'NISSAN':'JAPANESE',
    'SKODA':'EUROPEAN',
    'TOYOTA':'JAPANESE',
    'VOLKSWAGEN':'EUROPEAN',
    'VOLVO':'EUROPEAN'
}

df['vendor'] = df.brand.map(vendor_dict)

In [79]:
set(test.vendor.unique()) == set(df.vendor.unique())

True

#### Владельцы

In [80]:
owner_dict = {
    '3 или более': 3., 
    '2\xa0владельца': 2.,
    '1\xa0владелец': 1.
}
test.Владельцы = test.Владельцы.map(owner_dict)

In [83]:
set(test.Владельцы.unique()) , set(df.Владельцы.unique())

({1.0, 2.0, 3.0}, {nan, 1.0, 2.0, 3.0})

#### Владение

In [258]:
def test_ownership_unification(x):
    try:
        digits = re.findall('\d+',x) 
        if len(digits) == 2: res = int(digits[0])*12 + int(digits[1])
        elif len(digits) == 1 and 'месяц' in x: res = int(digits[0])
        elif len(digits) == 1 and 'месяц' not in x: res = int(digits[0])*12
    except Exception: res = 0   
    return res

test.Владение.apply(test_ownership_unification).unique()

array([  0,  38,  11,  49,  28,   1,  46,  30,  76,   2,   4,  22,  31,
        95,  89,  23,  25,  84,  50,  32,  29, 155,  71,  35,  37,  13,
        12,  19,   3,  17,  14,  92,  40,  18, 152,  73,  83,  64,  41,
        74,  54,  42,  20,  21,  47,  26,  36,  65,  33,  81,   9,  15,
        51,   8,  72,  16,  62,  34,  44,  53,  61,  58,  10,  96,  82,
        56,  94,  70, 125,  43,  87,  69,  45,   6,  79,  98, 100,  27,
        24,  52,  57, 344, 141, 199,  78,  68, 102, 151,  97,  86,  85,
       129,  77,   5, 178,  39,  99,  60, 211, 135, 157, 109, 113,   7,
       119, 110,  55, 148, 120, 111, 105, 115,  59, 101,  66,  91,  88,
        90,  48, 124, 112, 106,  67, 136, 143, 132, 123, 146, 108, 126,
        63, 190, 104, 140, 118, 137,  93, 139, 114, 149,  80, 134, 128,
        75, 130, 410, 270, 121, 142, 103, 280, 169, 144, 164, 250, 107,
       173, 127, 154, 216, 166, 208, 117, 217, 333, 153, 156, 167, 193,
       236, 131, 174, 191, 180, 138, 248, 150, 122, 171, 183, 15

In [259]:
def train_ownership_unification(x):
    tmp = json.loads(x.replace("'",'"'))  if x==x else {'year': 2020, 'month': 9}
    res = (2020 - tmp['year'])*12 + tmp['month'] - 9
    if res<0: res = 0
    return res

df.Владение.apply(train_ownership_unification).unique()

array([  0, 117,  39, 203,  13,  97,  71,  90,  11,   5,  46,  27,  30,
        14,  20, 316,  25,  37,  16, 111,  68,   4, 357,  44,  10,  81,
       175, 273,  42,  76,  49,  21,   8,  29,  73, 112, 162, 147, 192,
        28,   6,  62,  47,  33,  36,  59,  34,  35,  26, 143, 164,  54,
       407,  75,  24,  66,  72,  93, 110, 135,  82,  12, 121,  48,  78,
         9,  50,  83,  89,  31,  55,  57,  40,  17,  32,  64, 149,  79,
        19,  87,  18,  63,  43,  22,  69,  38,  15,  67, 122,   7,  23,
        53, 127,  65,  94,  85,  92,  86,  70, 178, 165,  52,  41, 106,
        84,  74, 108,  56,  99,  60, 105, 217, 144, 134,  95, 168,  77,
        88, 170, 102, 109, 255, 171, 163, 123, 131,  91,  45, 156,  96,
       104,  51, 114, 116, 103, 139, 120,  61, 119, 159, 137, 118, 201,
       126, 115, 128,  58, 221, 213, 150,  80, 146, 124, 136, 216, 100,
       157, 145, 113, 101, 519, 132, 125, 198, 133, 177, 141, 167, 107,
       267, 218,  98, 308, 138, 153, 184, 337, 191, 151, 189, 18

#### ПТС

In [85]:
test.ПТС.unique()

array(['Оригинал', 'Дубликат', nan], dtype=object)

In [92]:
PTS_dict = {
    'ORIGINAL': 'Оригинал', 
    'DUPLICATE': 'Дубликат'
}
df.ПТС = df.ПТС.map(PTS_dict)

In [94]:
set(test.ПТС.unique()) ==  set(df.ПТС.unique())

True

#### Привод

In [95]:
set(test.Привод.unique()) == set(df.Привод.unique())

True

#### Руль

In [99]:
wheel_dict = {
    'LEFT':'Левый', 
    'RIGHT':'Правый'}

df.Руль = df.Руль.map(wheel_dict)

In [101]:
set(test.Руль.unique()) == set(df.Руль.unique())

True

### Стандартизация датасетов

### Анализ распределения цены в 5(10) топ моделях

In [22]:
for df_name in  datasets_to_train:
    try:
        df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
    except Exception:
        df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
    df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
    
    print(df.columns)
#     models_for_distribution = df.model_name.value_counts().index[:5].to_list()
#     print(models_for_distribution)

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'],
      dtype='object')
Index(['Unnamed: 0', 'car_url', 'bodyType', 'brand', 'color',
       'complectation_dict', 'description', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage',
       'modelDate', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime',
       'price', 'priceCurrency', 'productionDate', 'sell_id', 'views',
       'date_added', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль',
       'Состояние', 'Таможня', 'region'],
      dtype='object')
Index(['bodyType', 'brand', 'car_url', 'col

In [20]:
models_for_distribution = df.model_name.value_counts().index[:5].to_list()
models_for_distribution

['Tiguan', 'X-Trail', 'Land Cruiser', 'Touareg', 'Polo']

### Соотношение курсового коэфициента с рапределением

In [10]:
'''for df_name in  datasets_to_train:
    
    try:
        df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
#         print('ok')
    except:
        df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
#         print('trouble')
#     print(df.brand.unique())
    df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
#     print(df.brand.unique())
    print(df_name,end=' ')
    try:
        print(date.fromtimestamp((df.parsing_unixtime.min())))
    except:
        print('no parsing_unixtime')
    print(df_name,end=' ')
    try:
        print(date.fromtimestamp((df.parsing_unixtime.max())))
    except:
        print('no parsing_unixtime')''';

## ==================================

In [27]:
def trfn(x):
    try:res = x.lower()   
    except Exception:res = x
    return res

df.bodyType.apply(trfn).unique()[:15]

array(['седан', 'универсал 5 дв.', 'хэтчбек 5 дв. sportback',
       'хэтчбек 3 дв.', 'хэтчбек 5 дв.', 'кабриолет', 'купе',
       'лифтбек sportback', 'лифтбек', 'седан long', 'внедорожник 5 дв.',
       'кабриолет roadster', 'седан 2 дв.', 'седан gran coupe',
       'компактвэн'], dtype=object)